In [13]:
from sagemaker import get_execution_role
import sagemaker
import boto3
from datetime import datetime

session = boto3.session.Session()
sagemaker_session = sagemaker.session.Session(boto_session=session)
# #same function
# sess = sage.Session()

role = 'arn:aws:iam::002224604296:role/service-role/AmazonSageMaker-ExecutionRole-20200402T124851'
aws_region = session.region_name
print(f'AWS region:{aws_region}')
dt = datetime.now()
dd = "{}-{}-{}-{}-{}-{}".format(dt.year,dt.month,dt.day,dt.hour,dt.minute,dt.second)

AWS region:us-east-1


In [8]:
model_path = "s3://{}/output".format(sagemaker.Session().default_bucket())
model_name = 'sagemaker-recsys-graph-inference'
s3_model_url = model_path+'/sagemaker-recsys-graph-2020-11-17-03-15-32-694/model.tar.gz'
# #same function：
# model_path = "s3://{}/output".format(sagemaker_session.default_bucket())
print(model_path)

s3://sagemaker-us-east-1-002224604296/output


In [9]:
serving_image = '002224604296.dkr.ecr.us-east-1.amazonaws.com/sagemaker-recsys-graph-inference'
model_name = 'sagemaker-recsys-graph-inference'
serving_container_def = {
    'Image': serving_image,
    'ModelDataUrl': s3_model_url,
    'Mode': 'SingleModel',
    'Environment': { 'SM_MODEL_DIR' : '/opt/ml/model',
                     'RESNET_ARCH': 'resnet50' # 'resnet50' or 'resnet101'
                   }
}

create_model_response = sagemaker_session.create_model(name=model_name, 
                                                       role=role, 
                                                       container_defs=serving_container_def)

print(create_model_response)

Using already existing model: sagemaker-recsys-graph-inference


sagemaker-recsys-graph-inference


In [16]:
endpoint_config_name=f'{model_name}-epc-{dd}'
print(endpoint_config_name)

sagemaker-recsys-graph-inference-epc-2020-11-25-8-13-27


In [17]:
epc = sagemaker_session.create_endpoint_config(
    name=endpoint_config_name, 
    model_name=model_name, 
    initial_instance_count=1, 
    instance_type='ml.p2.xlarge')
print(epc)

sagemaker-recsys-graph-inference-epc-2020-11-25-8-13-27


In [18]:
endpoint_name=f'{model_name}-endpoint'
print(endpoint_name)

sagemaker-recsys-graph-inference-endpoint


In [19]:
ep=sagemaker_session.create_endpoint(endpoint_name=endpoint_name, config_name=endpoint_config_name, wait=True)
print(ep)

-------------------!sagemaker-recsys-graph-inference-endpoint


In [26]:
import base64
import json

data = {"instance":'人生赢家千亿媳妇被亏橡胶假人？53岁张曼玉一笑倾城尴尬所有人'}
client = boto3.client('sagemaker-runtime')

body=json.dumps(data).encode('utf-8')

# print(body)
    
response = client.invoke_endpoint(EndpointName=endpoint_name,
                                  ContentType="application/json",
                                  Accept="application/json",
                                  Body=body)

# print(response[''])
body=response['Body'].read()
print(body)
# msg=body.decode('utf-8')

ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint sagemaker-recsys-graph-inference-endpoint of account 002224604296 not found.

In [25]:
boto3.client('sagemaker').delete_endpoint(EndpointName=endpoint_name)
boto3.client('sagemaker').delete_endpoint_config(epc)

ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "arn:aws:sagemaker:us-east-1:002224604296:endpoint/sagemaker-recsys-graph-inference-endpoint".